## Training

In [1]:
TRAIN_MODEL = True
if TRAIN_MODEL:
    import ivy
    import os

    class Regressor(ivy.Module):
        def __init__(self, input_dim, output_dim, is_training=True):
            self.linear = ivy.Linear(input_dim, output_dim)
            self.dropout = ivy.Dropout(0.5, training=is_training)
            ivy.Module.__init__(self)

        def _forward(self, x, ):
            x = ivy.sigmoid(self.linear(x))
            x = self.dropout(x)
            return x

    ivy.set_backend('torch')  # set backend to PyTorch

    model = Regressor(input_dim=3, output_dim=1)
    optimizer = ivy.Adam(1e-4)

    # generate some random data
    x = ivy.random.random_normal(shape=(100, 3))
    y = ivy.random.random_normal(shape=(100, 1))

    def loss_fn(pred, target):
        return ivy.mean((pred - target)**2)

    for epoch in range(50):
        # forward pass
        pred = model(x)

        # compute loss and gradients
        loss, grads = ivy.execute_with_gradients(lambda v: loss_fn(pred, y), model.v)

        # update parameters
        model.v = optimizer.step(model.v, grads)

        # print current loss
        if (epoch+1) % 10 == 0:
            print(f'Epoch: {epoch + 1:2d} --- Loss: {ivy.to_numpy(loss).item():.5f}')
            
    print('Finished training!')
    
    model_dir = "./artifacts"
    ivy_model = "weights.pkl"
    model_file = os.path.join(model_dir, ivy_model)
    
    if not os.path.exists(model_dir): os.makedirs(model_dir)
    model.v.cont_to_disk_as_pickled(model_file)

Epoch: 10 --- Loss: 1.43195
Epoch: 20 --- Loss: 1.18419
Epoch: 30 --- Loss: 1.19950
Epoch: 40 --- Loss: 1.46237
Epoch: 50 --- Loss: 1.41681
Finished training!


In [2]:
%%writefile artifacts/regressor.py
import ivy

class Regressor(ivy.Module):
    def __init__(self, input_dim, output_dim, is_training=True):
        self.linear = ivy.Linear(input_dim, output_dim)
        self.dropout = ivy.Dropout(0.5, training=is_training)
        ivy.Module.__init__(self)

    def _forward(self, x, ):
        x = ivy.sigmoid(self.linear(x))
        x = self.dropout(x)
        return x
    
input_dim = 3
output_dim = 1
backend = 'torch'

Writing artifacts/regressor.py


# Start serving our model
since this command will start the server and block the terminal, waiting for requests, this will need to be ran in the background on a separate terminal

In [ ]:
!mlserver start .

# Send test inference request

In [2]:
import requests
import numpy as np

from mlserver.types import InferenceRequest
from mlserver.codecs import NumpyCodec

x_0 = np.random.randn(4,3).astype(np.float32)
inference_request = InferenceRequest(
    inputs=[
        NumpyCodec.encode_input(name="X", payload=x_0)
    ]
)

endpoint = "http://localhost:8080/v2/models/ivy-regressor/infer"
response = requests.post(endpoint, json=inference_request.dict())

response.json()

{'model_name': 'ivy-regressor',
 'id': '954c6953-f6c9-46ea-a287-2792c9fba87d',
 'parameters': {},
 'outputs': [{'name': 'output-0',
   'shape': [4, 1],
   'datatype': 'FP32',
   'parameters': {'content_type': 'np'},
   'data': [0.3102598488330841,
    0.7707526087760925,
    0.986264169216156,
    0.13599324226379395]}]}